In [52]:
import pandas as pd
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv

In [53]:
# Load the password for the databse connection
load_dotenv()
DB_PW = os.getenv('DB_PW')

# Define the connection parameters
username = 'root'
password = DB_PW
host = 'localhost'
port = '3306'
database = 'sakila'

# Create the engine
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

In [54]:
def rentals_month(month, year, engine):
    query = text(f"""
    SELECT customer_id, inventory_id FROM rental
    WHERE MONTH(rental_date) = :month AND YEAR(rental_date) = :year;
    """)
    with engine.connect() as connection:
        rentals = pd.read_sql(query, connection, params={"month": month, "year": year})
    return rentals

In [55]:
rentals_may_2005 = rentals_month(5, 2005, engine)
rentals_june_2005 = rentals_month(6, 2005, engine)

In [56]:
def rental_count_month(df_rentals, month, year):
    df_rentals_grouped = df_rentals.groupby('customer_id').count().sort_values('inventory_id', ascending=False)
    df_rentals_out = pd.DataFrame({'customer_id': df_rentals_grouped.index, f'rentals_{month}_{year}': df_rentals_grouped['inventory_id']}).reset_index(drop=True)
    return df_rentals_out

In [57]:
df_rentals_out_may_2005 = rental_count_month(rentals_may_2005, 5, 2005)
df_rentals_out_june_2005 = rental_count_month(rentals_june_2005, 6, 2005)

In [58]:
def compare_rentals(df_1, df_2):
    df_compare = pd.merge(df_1, df_2, on='customer_id', how='outer')
    df_compare['difference'] = df_compare[f'rentals_5_2005'] - df_compare[f'rentals_6_2005']
    return df_compare

In [59]:
df_compare = compare_rentals(df_rentals_out_may_2005, df_rentals_out_june_2005)

In [60]:
df_compare.head()

,customer_id,rentals_5_2005,rentals_6_2005,difference
0,197,8.0,8.0,0.0
1,109,7.0,5.0,2.0
2,506,7.0,5.0,2.0
3,53,6.0,5.0,1.0
4,269,6.0,3.0,3.0
